In [1]:
pip install --upgrade gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.9/946.9 kB 40.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.106.1
    Uninstalling openai-1.106.1:
      Successfully uninstalled openai-1.106.1
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.12.1
    Uninstalling gradio_client-1.12.1:
      Successfully uninstalled gradio_client-1.12.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.44.1
    Uninstalling gradio-5.44.1:
      Successfully uninstalled gradio-5.44.1


In [2]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


In [27]:
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
from google.colab import userdata
import os

In [28]:
groq_api_key=userdata.get("GROQ_API_KEY")
groq_base_url="https://api.groq.com/openai/v1"

In [29]:
groq=OpenAI(
    base_url=groq_base_url,
    api_key=groq_api_key
)

In [30]:
### Loading LinkedIn Profile as pdf
loader=PdfReader("/content/Profile (2).pdf")
linkedin=""
for page in loader.pages:
  text=page.extract_text()
  if text:
    linkedin +=text

In [7]:
print(linkedin)

   
Contact
Dunya pur
03239917025  (Mobile)
as29624041@gmail.com
www.linkedin.com/in/muhammad-
asif-ambitious  (LinkedIn)
Top Skills
Data Analysis & Visualization
Model Deployment (Streamlit / Flask)
Statistical Data Analysis
Certifications
Deep Learning - Artificial Neural
Networks with TensorFlow
Generative AI with Large Language
Models
Generative AI Language Modeling
with Transformers
Generative AI Language Modeling
with TransformersMuhammad Asif
SOFTWARE ENGINEERING STUDENT | DATA SCIENCE
ENTHUSIAST | PYTHON | ML | ANN | GEN AI
Dunyapur, Punjab, Pakistan
Summary
Data Analyst | Machine Learning Enthusiast | Software Engineer
I’m a Software Engineer with a strong passion for Data Science, AI,
and Machine Learning. With expertise in Python, Pandas, Seaborn,
SQL, and Data Visualization, I specialize in extracting insights from
data and building impactful solutions.
Skilled in Data Analysis, Visualization, and Predictive Modeling
Experienced in Machine Learning, Neural Networks, and Dee

In [31]:
with open("/content/summary.txt","r", encoding="utf-8") as f:
  summary=f.read()

In [32]:
name="Muhammad Asif"

In [33]:
system_prompt=f"You are acting as {name}.You are answering questions on {name}`s website, \
particularly questions related to {name}`s career, background, skills and experience. \
your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given summary of {name}`s background and LinkedIn profile which you can use to answer the questions. \
Be professional and engaging, as if talking to a potential client or future employer who came accross the website. \
If you do not know the answer, Say so."

system_prompt +=f"\n\n## Summary: {summary}\n\n LinkedIn Profile: {linkedin}\n\n"
system_prompt +=f"with this context, please chat with user, always staying in character as {name}"


In [12]:
print(system_prompt)

You are acting as Muhammad Asif.You are answering questions on Muhammad Asif`s website, particularly questions related to Muhammad Asif`s career, background, skills and experience. your responsibility is to represent Muhammad Asif for interactions on the website as faithfully as possible. You are given summary of Muhammad Asif`s background and LinkedIn profile which you can use to answer the questions. Be professional and engaging, as if talking to a potential client or future employer who came accross the website. If you do not know the answer, Say so.

## Summary: About
Data Analyst | Machine Learning Enthusiast | Software Engineer

I’m a Software Engineer with a strong passion for Data Science, AI, and Machine Learning. With expertise in Python, Pandas, Seaborn, SQL, and Data Visualization, I specialize in extracting insights from data and building impactful solutions.

Skilled in Data Analysis, Visualization, and Predictive Modeling
Experienced in Machine Learning, Neural Networks,

In [35]:
groq_model="openai/gpt-oss-20b"

In [36]:
def chat(message, history):
  # Construct messages list with only 'role' and 'content' for history
  messages=[{"role":"system","content":system_prompt}]
  if history:
    for turn in history:
      if len(turn) == 2:  # Ensure the turn has both user and assistant messages
        human, assistant = turn
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
      # Optionally, handle cases where a turn might be incomplete (e.g., an empty list)
      # else:
      #   print(f"Skipping incomplete turn in history: {turn}")

  messages.append({"role":"user","content":message})

  response=groq.chat.completions.create(model=groq_model, messages=messages)
  return response.choices[0].message.content

In [15]:
gr.ChatInterface(chat, type="messages").launch(share=False,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.


In [16]:
!pip install pydantic

In [37]:
# Creating a pydantic model for evaluation
from pydantic import BaseModel
#schema
class Evaluation(BaseModel):
  is_acceptable: bool
  feedback:str

In [38]:
evaluator_system_prompt=f"You are an evaluator that decides wether response to a question is acceptable \
You are provided a conversation with a user and agent.your task is to evaluate wether agent`s latest response is acceptable \
the agent is playing the role of {name} and is representing {name} on thier website. \
The agent has been instructed to be professionaland engaging, as if talking to potential client or future employer who came accross the website. \
the agent has been provided with context on {name} in the form of thier summary and linkedin profile details here is the information"

evaluator_system_prompt +=f"\n\nSummary:\n{summary}\n\n ## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt +=f"with this context, please evaluate the latest response, replying with wether the response is acceptable or not"

In [39]:
def evaluator_user_prompt(reply, message, history):
  user_prompt=f"Here`s the conversation between the user and agent: \n\n"
  if history:
    for turn in history:
      if len(turn) == 2:  # Ensure the turn has both user and assistant messages
        human, assistant = turn
        user_prompt.append({"role": "user", "content": human})
        user_prompt.append({"role": "assistant", "content": assistant})
      # Optionally, handle cases where a turn might be incomplete (e.g., an empty list)
      # else:
      #   print(f"Skipping incomplete turn in history: {turn}")
        user_prompt.append("\n\n")
  user_prompt +=f"Here`s the latest mesage from the user: \n\n{message}\n\n"
  user_prompt +=f"Here`s the latest response from the agent: \n\n{reply}\n\n"
  user_prompt +=f"please evaluate the response, replying with wether it is acceptable and your feedback."
  return user_prompt

In [40]:
gemeini_api_key=userdata.get("GOOGLE_API_KEY")
gemini_base_url="https://generativelanguage.googleapis.com/v1beta/openai/"

In [41]:
gemini=OpenAI(
    base_url=gemini_base_url,
    api_key=gemeini_api_key
)

In [47]:
def evaluate(reply, message, history) -> Evaluation:
  messages=[{"role":"system","content":evaluator_system_prompt}] + [{"role":"user","content":evaluator_user_prompt(reply, message, history)}]
  response=gemini.beta.chat.completions.parse(model="gemini-2.5-pro", messages=messages, response_format=Evaluation)
  return response.choices[0].message.parsed

In [43]:
def chat(message,history):
  system=system_prompt
  messages=[{"role":"system","content":system}]
  if history:
    for turn in history:
      if len(turn) == 2:  # Ensure the turn has both user and assistant messages
        human, assistant = turn
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
      # Optionally, handle cases where a turn might be incomplete (e.g., an empty list)
      # else:
      #   print(f"Skipping incomplete turn in history: {turn}")

  messages.append({"role":"user","content":message})

  response=groq.chat.completions.create(model=groq_model, messages=messages)
  reply=response.choices[0].message.content

  evaluation=evaluate(reply,message,history)

  if evaluation.is_acceptable:
    print("Passed Evaluation-Returning Reply")
  else:
    print("Failed Evaluation-retrying")
    print(evaluation.feedback)
    reply=rerun(reply,message,history,evaluation.feedback)
  return reply

In [44]:
def rerun(reply,message,history,feedback):
  updated_system_prompt=system_prompt+f"\n\n ## previous answer rejected\n you just tried to reply but quality control reject your reply"
  updated_system_prompt +=f"\n##Your attempted answer: \n{reply}\n\n"
  updated_system_prompt +=f"## Reason for rejection: \n{feedback}\n\n"

  messages=[{"role":"system","content":updated_system_prompt}]
  if history:
    for turn in history:
      if len(turn) == 2:  # Ensure the turn has both user and assistant messages
        human, assistant = turn
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
      # Optionally, handle cases where a turn might be incomplete (e.g., an empty list)
      # else:
      #   print(f"Skipping incomplete turn in history: {turn}")

  messages.append({"role":"user","content":message})
  response=groq.chat.completions.create(model=groq_model, messages=messages)
  return response.choices[0].message.content

In [48]:
gr.ChatInterface(chat, type="messages").launch(share=False,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Passed Evaluation-Returning Reply
Passed Evaluation-Returning Reply
Passed Evaluation-Returning Reply
Keyboard interruption in main thread... closing server.
